In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE

audio_features_df = pd.read_csv("C:/Users/kakol/freelance/6/audio_features.csv")
bert_embeddings_df = pd.read_csv("C:/Users/kakol/freelance/6/bert_embeddings.csv")

# Merge datasets on Participant_ID
merged_df = pd.merge(audio_features_df, bert_embeddings_df, on="Participant_ID")

# Drop unnecessary columns
merged_df_cleaned = merged_df.drop(columns=["audio_name", "audio_class", "Label_y", "Participant_ID"])

# Extract features (X) and labels (y)
X = merged_df_cleaned.drop(columns=["Label_x"]).values  # Features (Audio + BERT Embeddings)
y = merged_df_cleaned["Label_x"].values  # Labels (Classification target)

# Standardize features (important for Transformer stability)
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Balance the dataset using SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Convert to PyTorch tensors
X_tensor = torch.tensor(X_resampled, dtype=torch.float32)
y_tensor = torch.tensor(y_resampled, dtype=torch.long)

# Split into train & test sets (40% for validation)
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.4, random_state=42)

# Create DataLoader for batch training
batch_size = 32
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Compute Class Weights for Imbalanced Data
class_weights = compute_class_weight('balanced', classes=np.unique(y_resampled), y=y_resampled)
class_weights = torch.tensor(class_weights, dtype=torch.float32)

# Define Transformer Model
class TransformerModel(nn.Module):
    def __init__(self, input_dim, output_dim, num_heads=4, ff_dim=512, num_layers=2):
        super(TransformerModel, self).__init__()

        self.embedding = nn.Linear(input_dim, ff_dim)
        encoder_layer = nn.TransformerEncoderLayer(d_model=ff_dim, nhead=num_heads, dim_feedforward=ff_dim)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        self.fc = nn.Linear(ff_dim, output_dim)
        self.dropout = nn.Dropout(0.6)  # Dropout to prevent overfitting

    def forward(self, x):
        x = self.embedding(x)  # Transform input to higher dimension
        x = x.unsqueeze(1)  # Reshape for Transformer (batch, seq_len, d_model)
        x = self.transformer_encoder(x)
        x = x.mean(dim=1)  # Global averaging
        x = self.dropout(x)
        return self.fc(x)  # Output layer

# Initialize Model
input_dim = X_train.shape[1]  # Number of input features (1756)
ff_dim = 512  # Feedforward hidden layer size
num_heads = 4  # Multi-head attention
num_layers = 2  # Number of Transformer encoder layers
output_dim = len(np.unique(y_resampled))  # Number of unique labels

model = TransformerModel(input_dim, output_dim, num_heads, ff_dim, num_layers)

criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)  # LR decay

num_epochs = 50
best_loss = float("inf")
patience = 5
counter = 0

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)

    model.eval()
    with torch.no_grad():
        val_outputs = model(X_test)
        val_loss = criterion(val_outputs, y_test)

    scheduler.step(val_loss)  # Reduce learning rate if needed

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}, Val Loss: {val_loss:.4f}")

    if val_loss < best_loss:
        best_loss = val_loss
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping triggered.")
            break

model.eval()
y_pred_list = []

with torch.no_grad():
    for batch_X, _ in test_loader:
        y_pred = model(batch_X)
        y_pred_classes = torch.argmax(y_pred, dim=1).cpu().numpy()
        y_pred_list.extend(y_pred_classes)

accuracy = accuracy_score(y_test.numpy(), y_pred_list)
print(f"Test Accuracy: {accuracy:.4f}")
print("Classification Report:")
print(classification_report(y_test.numpy(), y_pred_list))


c:\Users\kakol\miniconda3\envs\my_env\Lib\site-packages\torch\nn\modules\transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
c:\Users\kakol\miniconda3\envs\my_env\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch [1/50], Loss: 1.1392, Val Loss: 1.2449
Epoch [2/50], Loss: 0.8706, Val Loss: 0.4625
Epoch [3/50], Loss: 0.0949, Val Loss: 0.8314
Epoch [4/50], Loss: 0.0327, Val Loss: 0.5032
Epoch [5/50], Loss: 0.0853, Val Loss: 1.5610
Epoch [6/50], Loss: 0.4473, Val Loss: 0.8386
Epoch [7/50], Loss: 0.0183, Val Loss: 0.4832
Early stopping triggered.
Test Accuracy: 0.8000
Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.83      0.83        12
           1       0.75      0.75      0.75         8

    accuracy                           0.80        20
   macro avg       0.79      0.79      0.79        20
weighted avg       0.80      0.80      0.80        20

